# Functions and curve fitting
This notebook discusses curve fitting with arbitrary functions. It is intended to cover the same material as the relevant notebook from the P2 Python workshop, but it goes into a bit more detail about what's going on. This notebook was first used as part of the optional P2 Python drop-in sessions.

-- Jonathan Taylor, Nov 2020

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

# Very brief recap on functions
A *function* in Python behaves very much like a mathematical function such as $sin(x)$. We provide certain numerical *arguments* ($x$, in this case) and the result of the function is another number. A reminder of the terminology associated with functions in Python: A **function** "takes" one or more **arguments** (sometimes also referred to as **parameters**) that it operates on, and it **returns** a **result**. We then **call** the function from our code, every time we want to run the function, **passing** it the particular arguments we want it to run with each time.

*Advanced note: some programming languages distinguish between a "pure function" (which acts just like a mathematical function) that takes arguments and returns a result, and a "routine" (which may look similar, but might not take any arguments or return a value at all, and can have "side effects" - for example, it might print out a message, plot a graph, save a file to disk, etc). For example, strictly speaking, `silly_curve_fit` (introduced later) is a routine rather than a pure function*

In [ ]:
# A function is *DEFINED* using the "def" keyword, followed by:
# - the name of the function
# - a list of the arguments it takes (in brackets) 
# - and finally the colon character
# Anything on subsequent lines that is indented is considered a part of the function
def sinc(x):
    # This is just a very simple function that returns the mathematical sinc function sin(x)/x.
    return np.sin(x)/x

In [ ]:
# We have defined our function, but have not actually caused the code to be run at all.
# Now we *CALL* our 'sinc' function, just like we would with a built-in function such as np.cos
print(sinc(4.0))

# Demonstrate that we can evaluate the function on a whole array of numbers (in the variable x),
# returning a whole array of numbers (in the variable y). We then plot it.
x = np.linspace(-20, 20, 200)
y = sinc(x)
plt.plot(x, y)

# Generating some data and defining a quadratic function

In [ ]:
# Generate some equally-spaced x coordinates
x = np.linspace(-10, 10, 21)
# Generate some y values in the form of a quadratic function, plus some random variation
y = 4.1 * x**2 + (np.random.random((x.shape))-0.5)*100

# Plot the data just to take a look at it
plt.plot(x, y, 'x')
plt.show()

In order to use `curve_fit`, we first need to define a (Python) function representing the (mathematical) function we want to use to fit our data. Note that, to use our function with `curve_fit`, the first argument to our function must be x. Subsequent arguments represent all the parameters for our mathematical function. These are constants that we don't yet know the value of, but that we want `curve_fit` to figure out for us.

For example, we can define a quadratic function $f(x) = a\,x\,^2$, where $a$ is a constant. Since we know that our data (above) should have that form, it makes sense to try and fit that function to our datapoints.

In [ ]:
def quadratic(x, a):
    return a * (x**2)

In [ ]:
# If we were just crudely trying to estimate the best fit curve manually,
# a naive way of doing it would be just to try plotting our function called 'quadratic',
# for different trial values of a. We see that a=4 seems to be a vaguely plausible fit
plt.plot(x, y, 'x')
plt.plot(x, quadratic(x, 2))
plt.show()

plt.plot(x, y, 'x')
plt.plot(x, quadratic(x, 3))
plt.show()

plt.plot(x, y, 'x')
plt.plot(x, quadratic(x, 4))
plt.show()

plt.plot(x, y, 'x')
plt.plot(x, quadratic(x, 5))
plt.show()

# A silly implementation of a curve_fit function
We could actually write a *function* that did some of that work for us! It would be a bit of a silly function, but we could do it. Here, we have written a function called `silly_curve_fit` that plots graphs for various different values of $a$. It's then up to you to look at the graphs and decide which one you like best.

The reason I have done this is to demonstrate that there is nothing particularly mysterious about the function curve_fit. We could write our own one if we wanted, like this one `silly_curve_fit`, using nothing more than the programming skills we have already learned in the P2 Python workshop.

However, there is one subtlety here that is a potential source of much confusion. When we call `silly_curve_fit`, the first argument we pass to it is the **name** of the other function we previously defined (`quadratic`). In that line of code that calls `silly_curve_fit`, we are not directly **calling** `quadratic` (using round brackets after the word 'quadratic'). We are just passing the **name** of the function `quadratic` as the first argument to the function `silly_curve_fit`. `silly_curve_fit` will then in turn call our function every time it wants to evaluate the function. `silly_curve_fit` codes our generalised **algorithm** (albeit a silly one!) for curve fitting, but we can run that algorithm again and again with different specific functions (not just our `quadratic` function) if we wish.

In [ ]:
# Define our function
def silly_curve_fit(function, xData, yData, p0):
    '''
        Arguments:
         function     The name of a function we want to fit to our data
         xData        An array containing the x coordinates of our datapoints
         yData        An array containing the x coordinates of our datapoints
         p0           The initial guess for the unknown parameter 'a' in our function
                      (we have named this p0 to be consistent with the parameter name
                       used in the real curve_fit function)

        Limitations:
         silly_curve_fit assumes that the function used for fitting only takes one single parameter.
    '''
    # Loop over a range of different values of 'a'
    for a in range(p0-4, p0+4):
        # For each different value, plot the raw data and the fitted function
        plt.title('Function plotted with a={0}'.format(a))
        plt.plot(xData, yData, 'x')
        plt.plot(xData, function(xData, a))
        plt.show()
    # After the loop has finished, ask the user to make their own decision
    print('Which of those looks like the best fit to you?')

In [ ]:
# Now actually call our function.
# This will plot the graphs and prompt us to make a judgement call about the best fit.
# Note that we could call silly_curve_fit again later, with a *different* fitting function
# (instead of 'quadratic') if we wanted to.
silly_curve_fit(quadratic, x, y, 3)

# Doing it properly, with `curve_fit`

`scipy.optimize.curve_fit` is called in a very similar manner to our `silly_curve_fit`, but it actually works out the true best fit parameters and returns them to us.

Note 1: `curve_fit` is unusual in that it returns **two** different values (separated by a comma). The first is the value(s) for the unknown parameters (in our case there is just one of these, `a`). The second is the *covariance*, which we have not covered in P2, but which gives a measure of the uncertainty in the unknown parameters. The covariance is actually a really powerful tool, and is discussed in a separate notebook at `examples/more_advanced_examples/curve-fitting-covariance-and-uncertainties.ipynb`

Note 2: for the final argument we are passing to `curve_fit` there is a common convention to write `p0=(1)` rather than just `(1)`. In general (subject to a few ground rules) you are allowed to write the name of an argument (`p0` in this case) as well as the value you are passing in for it. Either approach is acceptable, but with curve_fit people often write `p0=(1)` (or equivalent) to help them remember what the `(1)` is actually for. That trick can be particularly helpful when you call a function that takes very large numbers of arguments.

Note 3: it doesn't actually matter whether you pass `p0` in the form `(1)`, `[1]` or even just `1`. You might see other examples elsewhere. It's a good habit to use either square or round brackets, though, because you *must* do that if your function has more than one unknown parameter.

In [ ]:
fit, cov = scipy.optimize.curve_fit(quadratic, x, y, p0=(1))

In [ ]:
# Check what value was returned for our 'a'.
# It's close to the true value of 4.1, but not exact (because there was noise added to our data)
print(fit)

In [ ]:
# Plot the best fit curve on top of our data
plt.plot(x, y, 'x')
plt.plot(x, quadratic(x, fit[0]))
plt.show()

# More complicated functions

What if we want to fit using a function that has more unknown parameters associated with it? For example, maybe we want to fit with the generalised quadratic $f(x) = a\,x\,^2 + b\,x + c$. We can make the following changes to our code above:
- Define a new function, taking more than one unknown parameter (plus the compulsory `x` as the first parameter)
- Pass initial guesses to `curve_fit` for *all* those parameters
- Adjust our code in recognition of the fact that the returned `fit` will be a list of more than one value

In [ ]:
def generalised_quadratic(x, a, b, c):
    return a * (x**2) + b * x + c

In [ ]:
fit, cov = scipy.optimize.curve_fit(generalised_quadratic, x, y, p0=(1,1,1))

In [ ]:
print(fit)

In [ ]:
# Plot the best fit curve on top of our data
plt.plot(x, y, 'x')
plt.plot(x, generalised_quadratic(x, fit[0], fit[1], fit[2]))
plt.show()

## Final physics note
If we were fitting real experimental data like this, we would have to make a decision about what our *physical model* of the experiment actually is, to decide whether it is more appropriate to fit with our generalised quadratic, our simple $x^2$ function, or indeed something else entirely. The more free parameters you introduce, the better your fit will be, but that doesn't mean that it's the most appropriate function to use for your fit, if you can't justify the function in terms of the physics of the experiment.